In [1]:
%run basics.ipynb

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

Using Theano backend.


In [2]:
df_tags = pd.read_csv('../coursera/data/ds_urls_shortnames_concepts_reviewed_2016_06_21.csv', encoding='utf-8')
print('len:', len(df_tags))
print('tags count:', df_tags['concept_tag'].count())
df_tags.head(2)

('len:', 940)
('tags count:', 940)


,id,course_shortname,url,concept_tag
0,ud257,ab-testing,https://www.udacity.com/course/ab-testing--ud2...,video_lecture
1,ud257,ab-testing,https://www.udacity.com/course/ab-testing--ud2...,in_depth


In [ ]:
#df = pd.read_csv('../coursera/data/learning_resource_table_2016_07_05.csv')
df = pd.read_csv('../coursera/data/learning_resource_w_slide_transcript_2016_07_07.csv')
print('len:', len(df))
df.head(2)

In [ ]:
df = pd.merge(df_tags, df, on='id', how='outer').groupby(by='id', as_index=False)
merged_df = pd.DataFrame(df.apply(lambda x: x.iloc[0]))
merged_df['tags'] = df.apply(lambda x: map(str, list(x['concept_tag'].values)))
print('len:', len(merged_df))
print(merged_df.columns)
merged_df.head(2)

In [ ]:
df = merged_df

In [ ]:
df.count()

In [ ]:
cols = ['title', 'subtitle', 'description', 'slide_text']
df['all-text'] = ''
for i in cols:
    df['all-text'] += df[i].astype('str')
    df['all-text'] += ' '
df['all-text'].tail(10)

In [ ]:
df['all-text'].apply(len).plot(kind='hist')
plt.xlabel('#char')
plt.show()
df['all-text'].apply(lambda x: len(x.split())).plot(kind='hist')
plt.xlabel('#words')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm, tqdm_pandas
import string
allowed_chars = set(string.ascii_lowercase) | {' '}

stemmer = PorterStemmer()
def text_cleanup(text):
    text = text.lower()
    #text = ' '.join((stemmer.stem(i.decode('utf-8')) for i in text.split()))
    if text == 'nan':
        text = ''
    text = filter(lambda x: x in allowed_chars, text.lower())
    text = ' '.join(filter(lambda x: len(x) > 4, text.split()))
    return text
    
df['all-text'] = df['all-text'].apply(text_cleanup)

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.1, min_df=5)
vectorized_text = vectorizer.fit_transform(df['all-text'].values)
print('tfidf shape:', vectorized_text.shape)

if False:
    from sklearn.decomposition import TruncatedSVD
    explained_var = list()
    comp_range = range(2, 3000, 300)
    shapes = list()
    for i in tqdm(comp_range):
        svd = TruncatedSVD(n_components=i, n_iter=100)
        tmp = svd.fit_transform(vectorized_text)
        explained_var.append(svd.explained_variance_.sum())
        shapes.append(tmp.shape[1])
    pd.DataFrame(columns=['number of components', 'explained variance', 'true dim'], 
                 data=zip(comp_range, explained_var, shapes)).plot(x='number of components', secondary_y=['true dim'])
    plt.show()
    svd = TruncatedSVD(n_components=3000, n_iter=100)
    vectorized_text = svd.fit_transform(vectorized_text)
else:
    vectorized_text = np.array(vectorized_text.todense())
print(type(vectorized_text))
print(vectorized_text.shape)
print(len(vectorized_text.shape))
tfidf_vals = pd.DataFrame(data=vectorized_text.copy().ravel())
tfidf_vals = tfidf_vals[tfidf_vals[0] > 0]
tfidf_vals.plot(kind='hist', bins=200)
plt.show()
tfidf_vals = pd.DataFrame(data=vectorized_text.max(axis=1).ravel())
print(len(tfidf_vals))
#tfidf_vals = tfidf_vals[tfidf_vals[0] > 0]
tfidf_vals.plot(kind='hist', bins=200)
plt.title('max tfidf vals')
plt.show()
tfidf_vals = pd.DataFrame(data=vectorized_text.min(axis=1).ravel())
print(len(tfidf_vals))
#tfidf_vals = tfidf_vals[tfidf_vals[0] > 0]
tfidf_vals.plot(kind='hist', bins=200)
plt.title('min tfidf vals')
plt.show()

In [ ]:
diffs = list()
for idx in range(vectorized_text.shape[0]):
    for jdx in range(vectorized_text.shape[0]):
        v1 = vectorized_text[idx, :].copy()
        v2 = vectorized_text[jdx, :].copy()
        
        sim = (v1 * v2).sum() / ( np.sqrt((v1**2).sum()) * np.sqrt((v2**2).sum()) )
        euc_dist = np.sqrt(((v1-v2)**2).sum())
        man_dist = np.abs(v1-v2).sum()
        bin_diff = (v2 > v1).sum()
        max_diff = (v2 - v1).max()
        mean_diff = (v2 - v1).mean()
        median_diff = np.median(v2 - v1)
        v1_max = v1.max()
        v2_max = v2.max()
        v1_min = v1.min()
        v2_min = v2.min()
        v1_mean = v1.mean()
        v2_mean = v2.mean()
        v1_median = np.median(v1)
        v2_median = np.median(v2)
        v1_count = (v1 > 0).sum()
        v2_count = (v2 > 0).sum()
        
        #cons_idx = (v2 > 0.) & (v1 > 0.) & (v2 - 0.4 > v1)
        #v1 = v1[cons_idx]
        #v2 = v2[cons_idx]
        
        diff = (v2 - v1).sum()
        diffs.append((idx, jdx, diff, sim, euc_dist, man_dist, 
                      bin_diff, max_diff, mean_diff, median_diff, 
                      v1_max, v2_max, v1_min, v2_min, v1_mean, v2_mean, v1_median, v2_median, v1_count, v2_count, v1, v2))

In [ ]:
diff_df = pd.DataFrame(columns=['idx', 'jdx', 
                                'vec-diff', 'cos-sim', 'euc-dist', 'man-dist', 'bin_diff', 
                                'max_diff', 'mean_diff', 'median_diff',
                                'v1_max', 'v2_max', 'v1_min', 'v2_min', 
                                'v1_mean', 'v2_mean', 'v1_median', 'v2_median', 
                                'v1_count', 'v2_count',
                                'v1', 'v2'], data=diffs)
diff_df['c1'] = map(lambda x: df.iloc[x]['title'], diff_df['idx'])
diff_df['c2'] = map(lambda x: df.iloc[x]['title'], diff_df['jdx'])
diff_df['c_id'] = map(lambda x: df.iloc[x]['id'], diff_df['idx'])
diff_df['prereq_id'] = map(lambda x: df.iloc[x][0], diff_df['jdx'])

In [ ]:
print(diff_df['vec-diff'].min())
print(diff_df['vec-diff'].max())
diff_df.to_pickle('coursera_clf.df')
diff_df.head(2)

In [ ]:
diff_df[diff_df['ground-truth']>0].head(10)

In [ ]:
diff_df.groupby(by='ground-truth').describe()

In [ ]:
diff_df[diff_df['diff']!=0]['diff'].plot(kind='hist', bins=200)
plt.show()
diff_df[diff_df['diff']>0]['diff'].plot(kind='hist', bins=200)
plt.show()

In [ ]:
def filter_top_prereq(df):
    s_df = df.sort_values(by='diff', ascending=True)
    return s_df[s_df['diff']>0.5].copy()
    #return s_df.iloc[-3:]

prereq = pd.DataFrame(diff_df.groupby(by='c1').apply(filter_top_prereq))
prereq

In [ ]:
prereq.head(2)

In [ ]:
df_prereq.head(2)

In [ ]:
def get_true_prereq_level((c_id, prereq_id), prereq_df):
    try:
        true_prereq = prereq_df.loc[c_id]
    except KeyError:
        return 0
    prereq_level = 0
    #print('---------')
    #print(df[df['id'] == c_id]['title'].iloc[0])
    #print(df[df['id'] == prereq_id]['title'].iloc[0])
    for idx, ids in enumerate(true_prereq.values):
        if isinstance(ids, float):
            continue
        #try:    
            #print(map(lambda x: df[df['id'] == x]['title'].iloc[0], ids.split()))
        #except:
        #    pass
        if prereq_id in ids.split():
            prereq_level = idx + 1
    #print(prereq_level)
    return prereq_level

In [ ]:
#prereq[['c_id', 'prereq_id']].iloc[:100].apply(get_true_prereq_level, args=(df_prereq,), axis=1)

In [ ]:
prereq['prereq_level'] = prereq[['c_id', 'prereq_id']].apply(get_true_prereq_level, args=(df_prereq,), axis=1)
prereq['prereq_level'].plot(kind='hist')
plt.show()
prereq[['prereq_level', 'diff']].groupby(by='prereq_level').describe()

In [ ]:
prereq[prereq['prereq_level'] == 0].sort_values(by='diff', ascending=False).head(100)

In [ ]:
prereq['rnd_prereq_id'] = np.random.choice(df['id'].values, size=len(prereq), replace=True)
prereq['rnd_prereq_id'].head(2)

In [ ]:
prereq['rnd_prereq_level'] = prereq[['c_id', 'rnd_prereq_id']].apply(get_true_prereq_level, args=(df_prereq,), axis=1)
prereq['rnd_prereq_level'].plot(kind='hist')
plt.show()

In [ ]:
prereq[prereq['prereq_level']==1]

In [ ]:
prereq[prereq['rnd_prereq_level']>0]

In [ ]:
a = '2D Game Development with libGDX'
b = 'Intro to Physics'
a_idx = df[df['title'] == a].index[0]
b_idx = df[df['title'] == b].index[0]

In [ ]:
def get_tfidf_df(vec, vectorizer):
    df = pd.DataFrame(columns=['tfidf-val'], data=vec)
    df['word'] = vectorizer.get_feature_names()
    return df

In [ ]:
a_df = get_tfidf_df(vectorized_text[a_idx, :], vectorizer)
a_df.sort_values(by='tfidf-val', ascending=False).head(10)

In [ ]:
a_df[a_df['tfidf-val']>0].sort_values(by='tfidf-val').head(10)

In [ ]:
b_df = get_tfidf_df(vectorized_text[b_idx, :], vectorizer)
b_df.sort_values(by='tfidf-val').tail(10)

In [ ]:
filt = (a_df['tfidf-val'] > 0) & (b_df['tfidf-val'] > 0)
a_df = a_df[filt]
b_df = b_df[filt]
a_df['diff'] = a_df['tfidf-val'] - b_df['tfidf-val']
a_df.sort_values(by='diff')